## 1. Install Required Libraries

In [ ]:
!pip install mediapipe opencv-python numpy

## 2. Import Libraries

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import os

print("✓ Libraries imported successfully!")
print(f"MediaPipe version: {mp.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 3. Define Helper Functions

In [ ]:
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    Args:
        a, b, c: Landmark points (x, y coordinates)
    Returns:
        angle: Angle in degrees
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

def extract_pose_features(landmarks):
    """
    Extract relevant features from pose landmarks for pullup detection
    """
    # Get coordinates for left side
    left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
    
    # Get coordinates for right side
    right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    # Calculate elbow angles
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    
    # Average elbow angle
    avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
    
    return [avg_elbow_angle]

print("✓ Helper functions defined!")

## 4. Pullup Counter Class

In [ ]:
class PullupCounter:
    def __init__(self, angle_threshold_up=90, angle_threshold_down=160):
        """
        Initialize pullup counter
        Args:
            angle_threshold_up: Elbow angle for 'up' position (top of pullup)
            angle_threshold_down: Elbow angle for 'down' position (bottom of pullup)
        """
        self.counter = 0
        self.stage = "down"  # Start in the 'down' position (arms extended)
        self.angle_threshold_up = angle_threshold_up
        self.angle_threshold_down = angle_threshold_down
        
    def update(self, elbow_angle):
        """
        Update counter based on elbow angle
        """
        # Up position
        if elbow_angle < self.angle_threshold_up:
            self.stage = "up"
        
        # Down position (and rep counted)
        if elbow_angle > self.angle_threshold_down and self.stage == "up":
            self.stage = "down"
            self.counter += 1
            
        return self.counter, self.stage
    
    def reset(self):
        """Reset counter"""
        self.counter = 0
        self.stage = "down"

print("✓ PullupCounter class ready!")

## 5. Main Pullup Counter Function (CPU Optimized for Windows)

In [ ]:
def run_pullup_counter(source='camera', video_path=None, duration=30):
    """
    Run pullup counter with webcam or video file
    Args:
        source: 'camera' or 'video' - input source type
        video_path: Path to video file (required if source='video')
        duration: Time duration in seconds (default 30, only for camera mode)
    """
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    
    print("🖥️  Using CPU mode (recommended for Windows)")
    
    # Initialize pullup counter
    pullup_counter = PullupCounter()
    
    # Setup video capture based on source
    if source.lower() == 'camera':
        cap = cv2.VideoCapture(0)
        use_duration = True
        print(f"🎥 Starting pullup counter with WEBCAM for {duration} seconds...")
        print("📍 Position yourself so your full body is visible")
    elif source.lower() == 'video':
        if not video_path or not os.path.exists(video_path):
            print(f"❌ Error: Video file not found: {video_path}")
            return None
        cap = cv2.VideoCapture(video_path)
        use_duration = False
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(f"🎥 Starting pullup counter with VIDEO: {os.path.basename(video_path)}")
        print(f"📹 Video Info: {total_frames} frames, {fps:.1f} FPS, ~{total_frames/fps:.1f}s duration")
        print("📹 Analyzing video... (Press 'Q' in video window to stop)")
    else:
        print(f"❌ Error: Invalid source '{source}'. Use 'camera' or 'video'")
        return None
    
    if not cap.isOpened():
        print(f"❌ Error: Cannot open {source}!")
        return None
    
    # Countdown for camera mode
    if source.lower() == 'camera':
        print("⏱️  Get ready in 3 seconds...\n")
        for i in range(3, 0, -1):
            print(f"{i}...")
            time.sleep(1)
    
    print("🏁 START!\n")
    
    start_time = time.time()
    frame_count = 0
    
    pose_config = {
        'min_detection_confidence': 0.5,
        'min_tracking_confidence': 0.5,
        'model_complexity': 1,  # 0=Lite, 1=Full, 2=Heavy
        'smooth_landmarks': True,
        'static_image_mode': False
    }
    
    with mp_pose.Pose(**pose_config) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("✓ Video processing completed!" if source == 'video' else "❌ Failed to grab frame")
                break
            
            frame_count += 1
            elapsed_time = time.time() - start_time
            remaining_time = max(0, duration - elapsed_time)
            
            if source.lower() == 'video' and frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100 if total_frames > 0 else 0
                print(f"⏳ Processing... {frame_count}/{total_frames} frames ({progress:.1f}%) - Pullups: {pullup_counter.counter}", end='\r')
            
            if use_duration and elapsed_time > duration:
                break
            
            if source.lower() == 'camera':
                frame = cv2.flip(frame, 1)
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                features = extract_pose_features(landmarks)
                elbow_angle = features[0]
                
                count, stage = pullup_counter.update(elbow_angle)
                
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )
                
                cv2.rectangle(image, (0,0), (350,150), (0,0,0), -1)
                cv2.putText(image, f'PULLUPS: {count}', (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
                stage_color = (0,255,255) if stage == "up" else (255,165,0)
                cv2.putText(image, f'STAGE: {stage.upper()}', (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, stage_color, 2)
                
                if use_duration:
                    cv2.putText(image, f'TIME: {int(remaining_time)}s', (10,120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
                else:
                    cv2.putText(image, f'TIME: {int(elapsed_time)}s', (10,120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            else:
                cv2.putText(image, 'NO PERSON DETECTED', (50,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            window_title = f'Pullup Counter ({source.upper()}) - Press Q to Exit'
            cv2.imshow(window_title, image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                print("\n⚠️  Stopped by user")
                break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if source.lower() == 'video':
        print() 
    
    actual_duration = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"🎉 ANALYSIS COMPLETE!")
    print(f"{'='*60}")
    print(f"  Source: {source.upper()}")
    if source.lower() == 'video':
        print(f"  Video: {os.path.basename(video_path)}")
    print(f"  Total Pullups: {pullup_counter.counter}")
    print(f"  Duration: {actual_duration:.1f} seconds")
    if actual_duration > 0:
        print(f"  Rate: {pullup_counter.counter / (actual_duration/60):.1f} pullups/minute")
    print(f"{'='*60}\n")
    
    return pullup_counter.counter

print("✓ Pullup counter function ready (Camera & Video support)!")

## 6. Run the Counter

In [ ]:
# Interactive option selector
print("="*60)
print("         PULLUP COUNTER - INPUT SOURCE SELECTOR")
print("="*60)
print("\nChoose input source:")
print("  1. Camera (Webcam)")
print("  2. Video File")
print("="*60)

# Set your choice here: 1 for Camera, 2 for Video
choice = 2

if choice == 1:
    print(f"\n🎥 Starting with CAMERA for 30 seconds...\n")
    result = run_pullup_counter(source='camera', duration=30)
    print(f"\n✅ Final Result: {result} pullups counted!")
    
elif choice == 2:
    # IMPORTANT: Change this path to your pullup video file
    video_path = r"D:\Sports-X\Test_data\pullups\1.mp4" # <-- CHANGE THIS
    
    if os.path.exists(video_path):
        print(f"\n📹 Starting with VIDEO: {video_path}\n")
        result = run_pullup_counter(source='video', video_path=video_path)
        print(f"\n✅ Final Result: {result} pullups counted!")
    else:
        print(f"\n❌ Error: Video file not found!")
        print(f"   Looking for: {video_path}")
        print(f"\n💡 Please update the 'video_path' variable in this cell.")
    
else:
    print("❌ Invalid choice! Please enter 1 or 2.")